In [5]:
pip install transformers sentence-transformers faiss-cpu datasets

   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   -- ------------------------------------- 0.8/13.8 MB 4.8 MB/s eta 0:00:03
   ---- ----------------------------------- 1.6/13.8 MB 4.7 MB/s eta 0:00:03
   ------- -------------------------------- 2.6/13.8 MB 4.6 MB/s eta 0:00:03
   ---------- ----------------------------- 3.7/13.8 MB 4.6 MB/s eta 0:00:03
   ------------ --------------------------- 4.5/13.8 MB 4.6 MB/s eta 0:00:03
   --------------- ------------------------ 5.5/13.8 MB 4.7 MB/s eta 0:00:02
   ------------------ --------------------- 6.6/13.8 MB 4.7 MB/s eta 0:00:02
   --------------------- ------------------ 7.3/13.8 MB 4.7 MB/s eta 0:00:02
   ------------------------ --------------- 8.4/13.8 MB 4.7 MB/s eta 0:00:02
   -------------------------- ------------- 9.2/13.8 MB 4.7 MB/s eta 0:00:01
   ----------------------------- ---------- 10.2/13.8 MB 4.7 MB/s eta 0:00:01
   -------------------------------- ------- 11.3/13.8 MB 4.7 MB/s eta 0:00:01
   -

In [6]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare corpus
corpus = [
    "Content of document 1.",
    "Content of document 2.",
]
corpus_embeddings = embedding_model.encode(corpus, convert_to_tensor=False)

# Create FAISS index
d = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(d)  # L2 distance metric
index.add(np.array(corpus_embeddings))
print(f"Added {index.ntotal} documents to the index.")

c:\Users\vikra\anaconda3\envs\test_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\vikra\anaconda3\envs\test_env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vikra\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In

Added 2 documents to the index.


In [7]:
query = "What is the content of document 1?"
query_embedding = embedding_model.encode(query, convert_to_tensor=False)

# Retrieve top 3 results
D, I = index.search(np.array([query_embedding]), k=3)  # Distances and indices
print(f"Top 3 documents: {I[0]}, Scores: {D[0]}")

Top 3 documents: [ 0  1 -1], Scores: [1.0664676e-01 3.2043615e-01 3.4028235e+38]


In [8]:
from transformers import pipeline

# Load a generative model (e.g., T5 or GPT)
generator = pipeline("text2text-generation", model="t5-small")

# Retrieve a document from the corpus and pass it to the generator
retrieved_docs = [corpus[i] for i in I[0]]
context = " ".join(retrieved_docs)
prompt = f"Based on the following documents: {context}. Answer the question: {query}"

# Generate a response
response = generator(prompt, max_length=100)
print(f"Generated Response: {response[0]['generated_text']}")

c:\Users\vikra\anaconda3\envs\test_env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vikra\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


Generated Response: Answer the question: What is the content of document 1?


In [9]:
def rag_pipeline(query, corpus, index, embedding_model, generator):
    # Retrieve relevant documents
    query_embedding = embedding_model.encode(query, convert_to_tensor=False)
    D, I = index.search(np.array([query_embedding]), k=3)
    retrieved_docs = [corpus[i] for i in I[0]]
    
    # Generate response based on retrieved documents
    context = " ".join(retrieved_docs)
    prompt = f"Based on the following documents: {context}. Answer the question: {query}"
    response = generator(prompt, max_length=100)
    return response[0]['generated_text']

# Example usage
response = rag_pipeline(query, corpus, index, embedding_model, generator)
print(f"RAG Response: {response}")

RAG Response: Answer the question: What is the content of document 1?


In [10]:
pip install fastapi uvicorn

  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.0 MB 4.8 MB/s eta 0:00:01
   ------------------------------- -------- 1.6/2.0 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 4.3 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
from fastapi import FastAPI

app = FastAPI()

@app.post("/rag")
def rag_api(query: str):
    response = rag_pipeline(query, corpus, index, embedding_model, generator)
    return {"response": response}

# Run the app
# uvicorn filename:app --reload